<a href="https://colab.research.google.com/github/mirchandani-mohnish/Musformer/blob/main/ConvAI_SimpleConvTasNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup


In [ ]:
import os
import shutil
import random
from google.colab import drive
from google.colab import auth


drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = '/content/drive/MyDrive/musdb18/musdb18 (2)/musdb18'
reduced_dataset_path = '/content/drive/MyDrive/musdb18/musdb18'


In [ ]:
!pip install musdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 21.8 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
np.float_ = np.float64
import musdb

MUS_DB_PATH = reduced_dataset_path

mus = musdb.DB(root=MUS_DB_PATH)
mus_train = musdb.DB(root=MUS_DB_PATH,subsets="train", split="train")
mus_valid = musdb.DB(root=MUS_DB_PATH,subsets="train", split="valid")
mus_test = musdb.DB(root=MUS_DB_PATH,subsets="test")
print(mus_train[0])
print(mus_test[0])

IndexError: list index out of range

In [ ]:
import torch

# Check if CUDA (NVIDIA GPU) is available
cuda_available = torch.cuda.is_available()

# Check if MPS (Apple Metal GPU) is available
mps_available = torch.backends.mps.is_available() if hasattr(torch.backends, 'mps') else False

# Set device to GPU if available, otherwise CPU
device = torch.device(
    "cuda" if cuda_available else
    "mps" if mps_available else
    "cpu"
)

print(f"Using device: {device}")

Using device: cuda


In [ ]:
%%capture
# Installing SpeechBrain via pip
BRANCH = 'develop'
!python -m pip install git+https://github.com/speechbrain/speechbrain.git@$BRANCH

# Clone SpeechBrain repository
!git clone https://github.com/speechbrain/speechbrain/

## Dataset

In [ ]:
# import json
# import torchaudio
# from speechbrain.utils.data_utils import get_all_files

# train_files = []
# valid_files = []
# test_files = []

# def create_json(json_file, mus_obj):

#   json_dict = {}
#   for i, track in enumerate(mus_obj):
#     if i % 10 == 0:
#       print(i)

#     file_name = track.name
#     file_path = track.path
#     file_rate = track.rate
#     # file_audio = track.audio
#     # file_vocal = track.targets['vocals'].audio
#     # print(file_name)
#     json_dict[file_name] = {
#               "file_path": file_path,
#               "file_index": i,
#               "rate": file_rate
#       }
#     # print(json_dict[file_name])

#     with open(json_file, mode="w") as json_f:
#         json.dump(json_dict, json_f, indent=2)

# # 80% for training
# create_json("train.json", mus_train)
# create_json("valid.json", mus_valid)
# create_json("test.json", mus_test)

In [ ]:
import json
import torchaudio
from speechbrain.utils.data_utils import get_all_files

train_files = []
valid_files = []
test_files = []

def create_json(json_file, files_path):

  json_dict = {}
  for track in get_all_files(files_path):

    # file_audio = track.audio
    # file_vocal = track.targets['vocals'].audio
    # print(file_name)
    file_name = track.split('/')[-1].split("_")[0]

    json_dict[file_name] = {
              "mix_path": files_path + '/' + file_name + '_mix.wav',
              "vocal_path": files_path + '/' + file_name + '_vocals.wav'
              # "rate": file_rate
    }
    # print(json_dict[file_name])

    with open(json_file, mode="w") as json_f:
        json.dump(json_dict, json_f, indent=2)

# 80% for training
create_json("train.json", reduced_dataset_path + "/combined_train")
# create_json("valid.json", mus_valid)
create_json("test.json", reduced_dataset_path+ "/combined_test")

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


## Simple ConvTasNet Model


In [ ]:
%%file models.py

import torch
from torch import nn
from torch.nn import Conv1d, ConvTranspose1d, LayerNorm, Sequential, Dropout, LeakyReLU

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()


        self.cnn = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=1, out_channels=10, kernel_size=17, stride=2,padding=8),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(p=0.2),
            torch.nn.Conv1d(in_channels=10, out_channels=5, kernel_size=9, stride=8, padding=4),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(p=0.2),
            torch.nn.Conv1d(in_channels=5, out_channels=40, kernel_size=5, stride=10, padding=2),
        )

    def forward(self, x):
        # Your code here. Aim for 1-2 lines.
        # x = x.permute(0,2, 1)
        out = self.cnn(x)
        return out


class Decoder(nn.Module):
    def __init__(self, inp_channels=40):
        super().__init__()


        self.cnn = torch.nn.Sequential(
            torch.nn.LeakyReLU(),
            torch.nn.ConvTranspose1d(in_channels=40, out_channels=5, kernel_size=5, stride=10,padding=2, output_padding=9),
            torch.nn.Dropout(p=0.2),
            torch.nn.ConvTranspose1d(in_channels=5, out_channels=10, kernel_size=9, stride=8, padding=4, output_padding=7),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(p=0.2),
            torch.nn.ConvTranspose1d(in_channels=10, out_channels=1, kernel_size=17, stride=2, padding=8, output_padding=1),
        )

    def forward(self, x):
        # Your code here. Aim for 1-2 lines.
        out = self.cnn(x)
        return out



Writing models.py


In [ ]:
%%file models.py

import torch
from torch import nn
from torch.nn import Conv1d, ConvTranspose1d, LayerNorm, Sequential, Dropout, LeakyReLU

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=10, kernel_size=17, stride=2, padding=8),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Conv1d(in_channels=10, out_channels=5, kernel_size=9, stride=8, padding=4),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Conv1d(in_channels=5, out_channels=40, kernel_size=5, stride=10, padding=2),
        )

    def forward(self, x):
        # Input shape: [batch, 1, time]
        # Output shape: [batch, 40, time_reduced]
        return self.cnn(x)


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.ConvTranspose1d(in_channels=40, out_channels=5, kernel_size=5, stride=10, padding=2, output_padding=9),
            nn.LeakyReLU(),
            nn.ConvTranspose1d(in_channels=5, out_channels=10, kernel_size=9, stride=8, padding=4, output_padding=7),
            nn.LeakyReLU(),
            nn.ConvTranspose1d(in_channels=10, out_channels=1, kernel_size=17, stride=2, padding=8, output_padding=1),
        )

    def forward(self, x):
        # Input shape: [batch, 40, time_reduced]
        # Output shape: [batch, 1, time]
        return self.cnn(x)

Overwriting models.py


In [ ]:
%%file hparams_convtasnet.yaml

# #################################
# Training a Convolutional Autoencoder
# #################################

# Seed needs to be set at top of yaml, before objects with parameters are made
seed: 1986
__set_seed: !!python/object/apply:torch.manual_seed [!ref <seed>]

output_folder: !ref ./results/Musdb/ConvTasNet/<seed>
save_folder: !ref <output_folder>/save
train_log: !ref <output_folder>/train_log.txt

# Path where data manifest files are stored
train_annotation: train.json
valid_annotation: valid.json
test_annotation: test.json

# The train logger writes training statistics to a file, as well as stdout.
train_logger: !new:speechbrain.utils.train_logger.FileTrainLogger
    save_file: !ref <train_log>


# Training Parameters
number_of_epochs: 10
batch_size: 5
lr_start: 0.0002
lr_final: 0.00002
max_samples: 12000

dataloader_options:
    batch_size: !ref <batch_size>


# Encoder
encoder: !new:models.Encoder

# Decoder
decoder: !new:models.Decoder


# The first object passed to the Brain class is this "Epoch Counter"
# which is saved by the Checkpointer so that training can be resumed
# if it gets interrupted at any point.
epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <number_of_epochs>

# Objects in "modules" dict will have their parameters moved to the correct
# device, as well as having train()/eval() called on them by the Brain class.
modules:
    encoder: !ref <encoder>
    decoder: !ref <decoder>

# This optimizer will be constructed by the Brain class after all parameters
# are moved to the correct device. Then it will be added to the checkpointer.
opt_class: !name:torch.optim.Adam
    lr: !ref <lr_start>

# This function manages learning rate annealing over the epochs.
# We here use the simple lr annealing method that linearly decreases
# the lr from the initial value to the final one.
lr_annealing: !new:speechbrain.nnet.schedulers.LinearScheduler
    initial_value: !ref <lr_start>
    final_value: !ref <lr_final>
    epoch_count: !ref <number_of_epochs>


loss: !name:speechbrain.nnet.losses.get_si_snr_with_pitwrapper


# This object is used for saving the state of training both so that it
# can be resumed if it gets interrupted, and also so that the best checkpoint
# can be later loaded for evaluation or inference.
checkpointer: !new:speechbrain.utils.checkpoints.Checkpointer
    checkpoints_dir: !ref <save_folder>
    recoverables:
        encoder: !ref <encoder>
        decoder: !ref <decoder>
        counter: !ref <epoch_counter>

Writing hparams_convtasnet.yaml


In [ ]:
%%file train.py

#!/usr/bin/env python3
"""Recipe for training a convolutional autoencoder with audioMNIST

To run this recipe, do the following:
> python train.py hparams.yaml

"""

import os
import sys
import torch
import torchaudio
from hyperpyyaml import load_hyperpyyaml
from speechbrain.utils.distributed import run_on_main
import speechbrain as sb
import numpy as np
np.float_ = np.float64
import musdb
import stempeg




# Brain class for signal reconstruction training
class SignalReconstructionBrain(sb.Brain):
    def compute_forward(self, batch, stage):
        """Runs all the computations that transform the input into the output.
        Arguments
        ---------
        batch : PaddedBatch
            This batch object contains all the relevant tensors for computation.
        stage : sb.Stage
            One of sb.Stage.TRAIN, sb.Stage.VALID, or sb.Stage.TEST.
        Returns
        -------
        predictions : Tensor
            Tensor that contains the predicted signal.
        """

        batch = batch.to(self.device)
        mix_sig = batch.mix_sig.data  # Access the tensor data
        # vocals_sig = batch.vocals_sig.data  # Access the tensor data

        # print(mix_sig.shape)
        # print(vocals_sig.shape)

        # # We first move the batch to the appropriate device.
        # # wavs, lens = batch.sig
        mix_sig = mix_sig.permute(0,2,1)
        # vocals_sig = vocals_sig.permute(0,2,1)
        print(mix_sig.shape)

        # print(vocals_sig.shape)
        # print(mix_sig)
        # mix_sig = mix_sig.unsqueeze(1)
        # mix_sig = mix_sig.unsqueeze(1)  # Shape: [batch, 1, time]

        # Forward pass through the encoder
        features = self.modules.encoder(mix_sig)  # Shape: [batch, channels, time]

        # Forward pass through the decoder
        predictions = self.modules.decoder(features)  # Shape: [batch, 1, time]

        # Remove the channel dimension
        predictions = predictions.squeeze(1)


        # wavs = wavs.unsqueeze(1)
        # # print(wavs.shape)
        # features = self.modules.encoder(wavs)
        # # print(features.shape)
        # predictions = self.modules.decoder(features).squeeze()
        # print(predictions.shape)


        # At test time, we store the samples into the output folder.
        # This way we can listen to the reconstructed samples.
        if stage == sb.Stage.TEST:
          save_path = self.hparams.output_folder+'/samples/'
          if not os.path.exists(save_path):
            os.makedirs(save_path)
          for i, id in enumerate(batch.id):
            save_file = save_path + id
            torchaudio.save(save_file, predictions[i].cpu().unsqueeze(0),16000)
            torchaudio.save(save_file.replace('.wav','_original.wav'), wavs[i].cpu(),16000)


        return predictions


    def compute_objectives(self, predictions, batch, stage):
        """Computes the loss given the predicted and targeted outputs.
        Arguments
        ---------
        predictions : tensor
            The output tensor from `compute_forward`.
        batch : PaddedBatch
            This batch object contains all the relevant tensors for computation.
        stage : sb.Stage
            One of sb.Stage.TRAIN, sb.Stage.VALID, or sb.Stage.TEST.
        Returns
        -------
        loss : torch.Tensor
            A one-element tensor used for backpropagating the gradient.
        """
        target, _ = batch.vocals_sig
        print("compute obj")
        target = target.permute(0,2,1)
        target = target.squeeze(1)
        print(target.shape)
        print(predictions.shape)

        # Loss computation.
        # Your code here. Aim for 1 line
        # loss = self.hparams.loss(target, predictions)
        # loss = torch.nn.MSELoss(predictions,target)

        criterion = torch.nn.MSELoss()  # Create the loss function
        loss = criterion(predictions, target)
        print(loss)

        return loss

    def on_stage_start(self, stage, epoch=None):
        """Gets called at the beginning of each epoch.
        Arguments
        ---------
        stage : sb.Stage
            One of sb.Stage.TRAIN, sb.Stage.VALID, or sb.Stage.TEST.
        epoch : int
            The currently-starting epoch. This is passed
            `None` during the test stage.
        """
        # Set up statistics trackers for this stage
        self.loss_metric = sb.utils.metric_stats.MetricStats(
            metric=sb.nnet.losses.mse_loss
        )

    def on_stage_end(self, stage, stage_loss, epoch=None):
        """Gets called at the end of an epoch.
        Arguments
        ---------
        stage : sb.Stage
            One of sb.Stage.TRAIN, sb.Stage.VALID, sb.Stage.TEST
        stage_loss : float
            The average loss for all of the dataset processed in this stage.
        epoch : int
            The currently-starting epoch. This is passed
            `None` during the test stage.
        """
        # Store the train loss until the validation stage.
        if stage == sb.Stage.TRAIN:
            self.train_loss = stage_loss

        # Summarize the statistics from the stage for record-keeping.
        else:
            stats = {
                "loss": stage_loss,
            }

        # At the end of validation...
        if stage == sb.Stage.VALID:

            old_lr, new_lr = self.hparams.lr_annealing(epoch)
            sb.nnet.schedulers.update_learning_rate(self.optimizer, new_lr)

            # The train_logger writes a summary to stdout and to the logfile.
            self.hparams.train_logger.log_stats(
                {"Epoch": epoch, "lr": old_lr},
                train_stats={"loss": self.train_loss},
                valid_stats=stats,
            )

            # Save the current checkpoint and delete previous checkpoints,
            self.checkpointer.save_and_keep_only(meta=stats, min_keys=["error"])

        # We also write statistics about test dataset to stdout and to the logfile.
        if stage == sb.Stage.TEST:
            self.hparams.train_logger.log_stats(
                {"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats=stats,
            )


def dataio_prep(hparams):



    # # Define audio pipeline
    # @sb.utils.data_pipeline.takes("file_path")
    # @sb.utils.data_pipeline.provides("mix_sig")
    # def audio_pipeline(file_path):
    #     print("just started")
    #     stems, rate = stempeg.read_stems(file_path)
    #     print('reached stems')

    #     mix_sig = torch.from_numpy(stems.sum(axis=0)).float()
    #     print(mix_sig)
    #     mix_sig = torchaudio.functional.resample(mix_sig, rate, 16000).squeeze(dim=0)
    #     print(mix_sig)
    #     mix_sig = mix_sig / mix_sig.abs().max()
    #     print(mix_sig)
    #     return mix_sig

    # @sb.utils.data_pipeline.takes("file_path")
    # @sb.utils.data_pipeline.provides("vocals_sig")
    # def target_pipeline(file_path):
    #     stems, rate = stempeg.read_stems(file_path)


    #     vocals_sig = torch.from_numpy(stems[0]).float()

    #     vocals_sig = torchaudio.functional.resample(vocals_sig, rate, 16000).squeeze(dim=0)

    #     vocals_sig = vocals_sig / vocals_sig.abs().max()
    #     print(vocals_sig)
    #     return vocals_sig

    # Define audio pipeline for the mixture (source)
    # @sb.utils.data_pipeline.takes("file_index","file_path")
    # @sb.utils.data_pipeline.provides("mix_sig", "vocals_sig")
    # def audio_pipeline(file_index,file_path):
    #     """Load the mixture and vocals signals from the MUSDB track."""
    #     # Load the track using musdb

    #     # Get the mixture and vocals signals
    #     mix_sig = torch.from_numpy(track.audio).float()  # Shape: [num_channels, num_samples]
    #     vocals_sig = torch.from_numpy(track.targets['vocals'].audio).float()  # Shape: [num_channels, num_samples]

    #     # Resample to 16 kHz
    #     mix_sig = torchaudio.functional.resample(mix_sig, track.rate, 16000)
    #     vocals_sig = torchaudio.functional.resample(vocals_sig, track.rate, 16000)

    #     # Normalize the waveforms to be between 0 and 1
    #     mix_sig = mix_sig / mix_sig.abs().max()
    #     vocals_sig = vocals_sig / vocals_sig.abs().max()

    #     return mix_sig, vocals_sig

    @sb.utils.data_pipeline.takes("mix_path","vocal_path")
    @sb.utils.data_pipeline.provides("mix_sig", "vocals_sig")
    def audio_pipeline(mix_path, vocal_path):
        """Load the mixture and vocals signals from .wav files."""
        # Load the mixture signal
        mix_sig, mix_rate = torchaudio.load(mix_path)  # Shape: [num_channels, num_samples]
        mix_sig = mix_sig.float()  # Convert to float32

        # Load the vocals signal (assuming the vocals file has a similar naming convention)
        vocals_sig, vocals_rate = torchaudio.load(vocal_path)  # Shape: [num_channels, num_samples]
        vocals_sig = vocals_sig.float()  # Convert to float32

        # Resample to 4 kHz (if necessary)
        # if mix_rate != 16000:
        mix_sig = torchaudio.functional.resample(mix_sig, mix_rate, 4000)
        # if vocals_rate != 16000:
        vocals_sig = torchaudio.functional.resample(vocals_sig, vocals_rate, 4000)

        # Normalize the waveforms to be between 0 and 1
        mix_sig = mix_sig / mix_sig.abs().max()
        vocals_sig = vocals_sig / vocals_sig.abs().max()


        # print(mix_rate)
        # print(mix_sig.shape)
        # print(vocals_rate)
        # print(vocals_sig.shape)

        mix_sig = _cap_length(mix_sig, hparams["max_samples"])
        vocals_sig = _cap_length(vocals_sig, hparams["max_samples"])

        return mix_sig.T, vocals_sig.T



    def _cap_length(waveform, max_samples):
        """Truncate or pad waveform to exactly max_samples."""
        num_samples = waveform.shape[1]  # Get current number of samples
        if num_samples > max_samples:
            return waveform[:, :max_samples]  # Truncate
        else:
            pad = max_samples - num_samples
            return torch.nn.functional.pad(waveform, (0, pad))  # Pad with zeros

    datasets = {}
    data_info = {
        "train": hparams["train_annotation"],
        # "valid": hparams["valid_annotation"],
        "test": hparams["test_annotation"],
    }
    hparams["dataloader_options"]["shuffle"] = False
    for dataset in data_info:
        datasets[dataset] = sb.dataio.dataset.DynamicItemDataset.from_json(
            json_path=data_info[dataset],
            dynamic_items=[audio_pipeline],
            output_keys=["id", "mix_sig", "vocals_sig"],
        )

    return datasets


# Recipe begins!
if __name__ == "__main__":
    # Reading command line arguments.
    hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])

    # Initialize ddp (useful only for multi-GPU DDP training).
    sb.utils.distributed.ddp_init_group(run_opts)

    # Load hyperparameters file with command-line overrides.
    with open(hparams_file) as fin:
        hparams = load_hyperpyyaml(fin, overrides)

    # Create experiment directory
    sb.create_experiment_directory(
        experiment_directory=hparams["output_folder"],
        hyperparams_to_save=hparams_file,
        overrides=overrides,
    )

    # Create dataset objects "train", "valid", and "test".
    datasets = dataio_prep(hparams)

    # Initialize the Brain object to prepare for mask training.
    brain = SignalReconstructionBrain(
        modules=hparams["modules"],
        opt_class=hparams["opt_class"],
        hparams=hparams,
        run_opts=run_opts,
        checkpointer=hparams["checkpointer"],
    )

    # The `fit()` method iterates the training loop, calling the methods
    # necessary to update the parameters of the model. Since all objects
    # with changing state are managed by the Checkpointer, training can be
    # stopped at any point, and will be resumed on next call.
    brain.fit(
        epoch_counter=brain.hparams.epoch_counter,
        train_set=datasets["train"],
        # valid_set=datasets["valid"],
        train_loader_kwargs=hparams["dataloader_options"],
        valid_loader_kwargs=hparams["dataloader_options"],
    )

    # Load the best checkpoint for evaluation
    test_stats = brain.evaluate(
        test_set=datasets["test"],
        min_key="error",
        test_loader_kwargs=hparams["dataloader_options"],
    )


Overwriting train.py


In [ ]:
# To start from scratch, you need to remove the output folder.
# Otherwise, speechbrain starts from the last valid checkpoint.
#!rm -rf ./results/AudioMNIST/Autoencoder/

!python train.py hparams_convtasnet.yaml --device "cpu"

speechbrain.utils.quirks - Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
speechbrain.utils.quirks - Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: ./results/Musdb/ConvTasNet/1986
speechbrain.core - Gradscaler enabled: `False`
speechbrain.core - Using training precision: `--precision=fp32`
speechbrain.core - Using evaluation precision: `--eval_precision=fp32`
speechbrain.core - SignalReconstructionBrain Model Statistics:
* Total Number of Trainable Parameters: 3.3k
* Total Number of Parameters: 3.3k
* Trainable Parameters represent 100.0000% of the total size.
speechbrain.utils.checkpoints - Would load a checkpoint here, but none found yet.
speechbrain.utils.epoch_loop - Going into epoch 1
  0% 0/2 [00:00<?, ?it/s]torch.Size([5, 1, 12000])
compute obj
torch.Size([5, 12000])
torch.Size([5, 12000])
tensor(0.0028, grad_fn=<MseLos

## Archives


In [ ]:
# !git clone https://github.com/pyenv/pyenv.git ~/.pyenv

# # Set the root for pyenv
# os.environ['PYENV_ROOT'] = os.path.expanduser("~/.pyenv")
# # Prepend pyenv's bin folder to PATH
# os.environ['PATH'] = os.environ['PYENV_ROOT'] + '/bin:' + os.environ['PATH']

# # Confirm pyenv is callable
# !pyenv --version
# !apt-get install libffi-dev
# !pyenv install 3.8.13
# !pyenv global 3.8.13
# !pyenv exec pip install spleeter
# !pyenv exec spleeter